In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os

def extract_frames(video_path, max_frames=50):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        resized_frame = cv2.resize(gray_frame, (128, 128))
        frames.append(resized_frame)
        count += 1
    cap.release()
    return np.array(frames)

def apply_dct(image):
    dct = cv2.dct(np.float32(image))
    normalized_dct = cv2.normalize(dct, None, 0, 255, cv2.NORM_MINMAX)
    return np.expand_dims(normalized_dct, axis=-1)

def load_dataset_from_videos(video_paths, labels, max_frames=50):
    data = []
    new_labels = []
    for i, video_path in enumerate(video_paths):
        if not os.path.exists(video_path):
            continue
        frames = extract_frames(video_path, max_frames)
        if len(frames) == 0:
            continue
        for frame in frames:
            try:
                dct_frame = apply_dct(frame)
                data.append(dct_frame)
                new_labels.append(labels[i])
            except Exception as e:
                continue
    if len(data) == 0:
        raise ValueError("No data available: check your video paths and frame extraction.")
    data = np.array(data)
    new_labels = np.array(new_labels)
    return data, new_labels

def build_cnn_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def prepare_ffpp_dataset(base_path):
    video_paths = []
    labels = []
    for label, folder in enumerate(['real', 'fake']):  # 'real' = 0, 'fake' = 1
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            print(f"Warning: {folder_path} does not exist.")
            continue
        for video_file in os.listdir(folder_path):
            if video_file.endswith('.mp4'):  # Adjust for your video format
                video_paths.append(os.path.join(folder_path, video_file))
                labels.append(label)
    return video_paths, labels

def train_cnn(data, labels):
    labels = to_categorical(labels, num_classes=2)
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)
    model = build_cnn_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Accuracy: {accuracy}')
    return model

base_path = "C:\\Users\\Pavan\\Downloads\\FF++"
video_paths, labels = prepare_ffpp_dataset(base_path)

try:
    data, labels = load_dataset_from_videos(video_paths, labels)
    if len(data) > 0:
        cnn_model = train_cnn(data, labels)
except ValueError as e:
    print(e)